# Community  Detection  In  Complex  Network  Using  Leader  Knowledge

> File contains code for SCAN and Girvan-Newmann algorithm and their performance evaluation

Work done by 
* Abhishek Kumar - 202IT001
* Nitin Sharma - 202IT017
* Mohd Asif Khan Khaisagi - 202IT013

## Library Import

In [ ]:
%%capture
!pip install cdlib
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import pandas as pd
import numpy as np
import random
from networkx.algorithms.components import connected_components
from cdlib import algorithms

## Dataset Imports

* American Football : http://konect.cc/networks/dimacs10-football/
> This network contains "American football games between Division IA colleges during regular season Fall 2000." Results are not included in the dataset, and neither is home/away information. 

* Zachary = http://konect.cc/networks/ucidata-zachary/
> This is the well-known and much-used Zachary karate club network. The data was collected from the members of a university karate club by Wayne Zachary in 1977. Each node represents a member of the club, and each edge represents a tie between two members of the club. The network is undirected. An often discussed problem using this dataset is to find the two groups of people into which the karate club split after an argument between two teachers. 

* Political books : http://konect.cc/networks/dimacs10-polbooks/
> his is "[a] network of books about US politics published around the time of the 2004 presidential election and sold by the online bookseller Amazon.com. Edges between books represent frequent copurchasing of books by the same buyers. The network was compiled by V. Krebs and is unpublished, but can found on Krebs' web site (http://www.orgnet.com/). Thanks to Valdis Krebs for permission to post these data on this web site."

In [ ]:
df_football = pd.read_csv('football.csv')
karate_graph = nx.karate_club_graph()
df_book = pd.read_csv('polbooks.csv')

## Testing

* We have used 3 testing measures : NMI, ARI, cluster purity
* Each of these 3 metrics accepts two values as parameters :
  * Truth Values : Actual cluster index of each node
  * Predicted Truth Value : Predicted cluster index of each node
* Truth values are analogous to Y_test of machine learning based models.
* Predicted Truth Values is analogous to Y_predicted of machine learning models. Our algorithm gives clusters as outputs and truth value is the cluster number (0, 1, 2, 3...) or (a, b, c,...) for each node showing which cluster does a particular node actually belongs to.
* After clusters are returned by our algorithm, we need to observe the result and give them suitable cluster index. This is similar to labeling in all clustering algorithms where upon forming clusters we give them class labels (eg : spam or ham, 0 or 1, true or false, etc).
* For higher performance, Actual and predicted truth values showing clustering of nodes should be overall as similar as possible.

In [ ]:
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn import metrics

def calculatePurity(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

def calculateNMI(labels_true, labels_pred) :
  return normalized_mutual_info_score(labels_true=labels_true, labels_pred=labels_pred)

def calculateARI(labels_true, labels_pred) :
  return adjusted_rand_score(labels_true=labels_true, labels_pred=labels_pred)

## Parameters in SCAN and Girvan newmann

* Parameters used in SCAN : 
  * g_original – a networkx/igraph object
  * epsilon – the minimum threshold to assigning cluster membership
  * mu – minimum number of neineighbors with a structural similarity that exceeds the threshold epsilon

* Parameters used in Girvan-newmann : 
  * g_original – a networkx/igraph object
  * level – the level where to cut the dendrogram


## Zachary

Code for testing Zachary database on SCAN and Girvan Newmann

In [ ]:
def getZachardyTruth(): # Get Actual Truth value for Zachary database
  zachardy_truth_value = [0]*34
  zachardy_0_node = [0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 16, 17, 19, 21]
  zachardy_1_node = [8, 9, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33] # after subtracting 1 to make node index start from 0

  for key in zachardy_1_node :
    zachardy_truth_value[key] = 1
  
  return zachardy_truth_value

Printing graph details

In [ ]:
karate_graph = nx.karate_club_graph()
print(nx.info(karate_graph))
print("Number of connected_components : ", nx.number_connected_components(karate_graph))

Name: Zachary's Karate Club
Type: Graph
Number of nodes: 34
Number of edges: 78
Average degree:   4.5882
Number of connected_components :  1


In [ ]:
zachary_truth = getZachardyTruth()

### Scan

* Using in built SCAN algorithm for creation of graph and then finding communities from it.

In [ ]:
karate_scan = algorithms.scan(karate_graph, 0.5, 2)
karate_scan.communities

[[0, 1, 2, 3, 7, 13, 17, 21, 12],
 [32, 8, 30, 33, 29, 23, 26, 27],
 [5, 6, 4, 16, 10],
 [24, 25, 31, 28]]

* Generating predicted truth values for each node relating them with a cluster index denoting the label of cluster that they belong to.

* Evaluate metrics


In [ ]:
zachardy_pred_value_scan = [0]*34

new_cluster_dic = { 0 : [0, 1, 2, 3, 7, 13, 17, 21, 12],
                    1 : [8, 30, 32, 33, 29, 23, 26, 27],
                    2 : [6, 4, 10, 5, 16],
                    3 : [25, 24, 31, 28]}

for key in new_cluster_dic.keys() :
  for val in new_cluster_dic[key] :
    zachardy_pred_value_scan[val] = key

print("NMI : ", calculateNMI(zachary_truth, zachardy_pred_value_scan))
print("ARI : ", calculateARI(zachary_truth, zachardy_pred_value_scan))
print("Purity : ", calculatePurity(zachary_truth, zachardy_pred_value_scan))

NMI :  0.38365064769070495
ARI :  0.19010221169948605
Purity :  0.8235294117647058


### Girvan Newman

* Using in built Girvan-newmann algorithm for creation of graph and then finding communities from it.

In [ ]:
karate_girvan = algorithms.girvan_newman(karate_graph,level=3)
karate_girvan.communities

[[32, 33, 2, 8, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
 [0, 1, 3, 7, 11, 12, 13, 17, 19, 21],
 [4, 5, 6, 10, 16],
 [9]]

* Generating predicted truth values for each node relating them with a cluster index denoting the label of cluster that they belong to.

* Evaluate metrics

In [ ]:
zachardy_pred_value_girvan = [0]*34

new_cluster_dic = { 1 : [32, 33, 2, 8, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
                    0 : [0, 1, 3, 7, 11, 12, 13, 17, 19, 21],
                    2 : [4, 5, 6, 10, 16],
                    3 : [9]}

for key in new_cluster_dic.keys() :
  for val in new_cluster_dic[key] :
    zachardy_pred_value_girvan[val] = key

print("NMI : ", calculateNMI(zachary_truth, zachardy_pred_value_girvan))
print("ARI : ", calculateARI(zachary_truth, zachardy_pred_value_girvan))
print("Purity : ", calculatePurity(zachary_truth, zachardy_pred_value_girvan))

NMI :  0.651560852017932
ARI :  0.6446027144804285
Purity :  0.9705882352941176


## Football

Code for testing Football database on SCAN and Girvan Newmann

In [ ]:
df_football = pd.read_csv('football.csv')
football_graph = nx.from_pandas_edgelist(df_football, source='from', target=' to')
print(nx.info(football_graph))
print("Number of connected_components : ", nx.number_connected_components(football_graph))

Name: 
Type: Graph
Number of nodes: 115
Number of edges: 613
Average degree:  10.6609
Number of connected_components :  1


#### Actual Truth Values for football dataset

In [ ]:
football_truth = [7, 0, 2, 3, 7, 3, 2, 8, 8, 7, 3, 10, 6, 2, 6, 2, 7, 9, 6, 1, 9, 8, 8, 
                  7, 10, 0, 6, 9, 11, 1, 1, 6, 2, 0, 6, 1, 5, 0, 6, 2, 3, 7, 5, 6, 4, 0, 11, 2, 
                  4, 11, 10, 8, 3, 11, 6, 1, 9, 4, 11, 10, 2, 6, 9, 10, 2, 9, 4, 11, 8, 10, 9, 
                  6, 3, 11, 3, 4, 9, 8, 8, 1, 5, 3, 5, 11, 3, 6, 4, 9, 11, 0, 5, 4, 4, 7, 1, 
                  9, 9, 10, 3, 6, 2, 1, 3, 0, 7, 0, 2, 3, 8, 0, 4, 8, 4, 9, 11]

### Scan

* Using in built SCAN algorithm for creation of graph and then finding communities from it.

In [ ]:
football_scan = algorithms.scan(football_graph, 0.5, 2)
football_scan.communities

[[54, 31, 18, 34, 61, 71, 99, 38, 26, 14, 12, 43, 85],
 [62, 27, 17, 56, 65, 70, 76, 95, 96, 87, 20, 113],
 [102, 74, 72, 52, 3, 40, 84, 5, 98, 10, 81, 107],
 [15, 13, 2, 32, 39, 60, 64, 100, 106, 6, 47],
 [78, 77, 22, 7, 51, 21, 8, 108, 111, 68],
 [1, 25, 33, 37, 45, 89, 103, 105, 109],
 [53, 49, 46, 67, 73, 83, 88, 110, 114],
 [75, 66, 44, 48, 86, 91, 92, 112, 57],
 [29, 19, 30, 35, 55, 79, 94, 101],
 [93, 0, 9, 16, 23, 41, 104, 4],
 [50, 24, 11, 28, 69, 90]]

* Generating predicted truth values for each node relating them with a cluster index denoting the label of cluster that they belong to.

* Evaluate metrics

In [ ]:
facebook_pred_vals_scan = [0]*115
new_cluster_dic = {
                    3: [14, 12, 26, 38, 43, 85, 54, 31, 34, 18, 61, 71, 99],
                    5: [3, 5, 52, 84, 98, 40, 72, 74, 81, 102, 107, 10],
                    6: [76, 27, 17, 56, 62, 65, 70, 95, 96, 20, 87, 113],
                    1: [47, 6, 32, 39, 60, 64, 100, 106, 15, 13, 2],
                    7: [7, 21, 22, 68, 77, 78, 108, 111, 8, 51],
                    9: [44, 48, 57, 66, 75, 86, 91, 92, 112],
                    8: [46, 49, 53, 67, 73, 83, 88, 110, 114],
                    4: [103, 33, 1, 25, 37, 45, 89, 105, 109],
                    10: [19, 29, 35, 55, 79, 94, 101, 30],
                    0: [0, 9, 4, 16, 23, 41, 93, 104],
                    2: [11, 24, 28, 50, 69, 90]
                   }


for key in new_cluster_dic.keys() :
  for val in new_cluster_dic[key] :
    facebook_pred_vals_scan[val] = key

print("NMI : ", calculateNMI(football_truth, facebook_pred_vals_scan))
print("ARI : ", calculateARI(football_truth, facebook_pred_vals_scan))
print("Purity : ", calculatePurity(football_truth, facebook_pred_vals_scan))

NMI :  0.9204052653227827
ARI :  0.8595850290406055
Purity :  0.9043478260869565


### Girvan Newman

* Using in built Girvan-newmann algorithm for creation of graph and then finding communities from it.

In [ ]:
football_girvan = algorithms.girvan_newman(football_graph, level=11)
football_girvan.communities

[[34, 99, 36, 38, 71, 42, 43, 12, 14, 18, 85, 54, 26, 61, 31],
 [64, 32, 2, 100, 6, 39, 106, 13, 47, 15, 82, 60],
 [98, 3, 5, 102, 40, 72, 74, 107, 10, 81, 52, 84],
 [96, 65, 70, 76, 17, 113, 20, 87, 56, 27, 62, 95],
 [68, 7, 8, 108, 77, 78, 111, 50, 51, 21, 22],
 [66, 75, 44, 48, 112, 80, 86, 57, 91, 92],
 [67, 73, 110, 46, 49, 114, 83, 53, 88, 58],
 [1, 33, 37, 103, 105, 45, 109, 89, 25],
 [0, 4, 104, 9, 41, 16, 23, 93],
 [35, 101, 79, 19, 30, 55, 29, 94],
 [69, 11, 24, 90, 28],
 [97, 59, 63]]

* Generating predicted truth values for each node relating them with a cluster index denoting the label of cluster that they belong to.

* Evaluate metrics

In [ ]:
facebook_pred_vals_girvan = [0]*115
new_cluster_dic = {
                    3: [34, 99, 36, 38, 71, 42, 43, 12, 14, 18, 85, 54, 26, 61, 31],
                    1: [64, 32, 2, 100, 6, 39, 106, 13, 47, 15, 82, 60],
                    5: [98, 3, 5, 102, 40, 72, 74, 107, 10, 81, 52, 84],
                    6: [96, 65, 70, 76, 17, 113, 20, 87, 56, 27, 62, 95],
                    7: [68, 7, 8, 108, 77, 78, 111, 50, 51, 21, 22],
                    9: [66, 75, 44, 48, 112, 80, 86, 57, 91, 92],
                    8: [67, 73, 110, 46, 49, 114, 83, 53, 88, 58],
                    11: [1, 33, 37, 103, 105, 45, 109, 89, 25],
                    0: [0, 4, 104, 9, 41, 16, 23, 93],
                    10: [35, 101, 79, 19, 30, 55, 29, 94],
                    2: [69, 11, 24, 90, 28],
                    4: [97, 59, 63],
                   }


for key in new_cluster_dic.keys() :
  for val in new_cluster_dic[key] :
    facebook_pred_vals_girvan[val] = key

print("NMI : ", calculateNMI(football_truth, facebook_pred_vals_girvan))
print("ARI : ", calculateARI(football_truth, facebook_pred_vals_girvan))
print("Purity : ", calculatePurity(football_truth, facebook_pred_vals_girvan))

NMI :  0.9213801197236223
ARI :  0.8845869070922053
Purity :  0.9304347826086956


## Polbooks

Code for testing Polbooks database on SCAN and Girvan Newmann

In [ ]:
df_book = pd.read_csv('polbooks.csv')
book_graph = nx.from_pandas_edgelist(df_book, source='from', target='to')
print(nx.info(book_graph))
print("Number of connected_components : ", nx.number_connected_components(book_graph))

Name: 
Type: Graph
Number of nodes: 105
Number of edges: 441
Average degree:   8.4000
Number of connected_components :  1


#### Actual Truth value for Book dataset

In [ ]:
books_truth = [2,0,0,0,2,0,2,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,
              0,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,
              0,0,2,0,2,0,0,2,0,0,0,0,0,0,0,1,1,1,1,1,1,1,
              1,1,1,2,1,1,1,1,1,1,2,0,1,1,1,1,1,1,1,1,1,1,
              1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2]

### Scan

* Using in built SCAN algorithm for creation of graph and then finding communities from it.

In [ ]:
book_scan = algorithms.scan(book_graph, 0.5, 2)
print(book_scan.communities)

[[12, 3, 8, 11, 14, 21, 9, 13, 40, 10, 15, 16, 33, 37, 38, 39, 55, 17, 42, 44, 32, 23, 41, 36, 27, 47, 54, 45, 26, 20, 24, 34, 35, 19, 43, 53, 48, 49, 57], [59, 60, 62, 63, 99, 66, 84, 30, 72, 73, 86, 88, 89, 74, 82, 83, 96, 97, 100, 31, 75, 76, 71, 93, 101, 78, 79, 91, 94, 81, 77, 61, 98, 95, 102, 87], [69, 64, 58, 51, 52, 65, 68], [4, 0, 1, 2, 5, 6], [104, 67, 103]]


* Generating predicted truth values for each node relating them with a cluster index denoting the label of cluster that they belong to.

* Evaluate metrics

In [ ]:
book_pred_vals_scan = [0]*105

new_cluster_dic = { 
                  0: [35, 34, 36, 37, 38, 39, 10, 33, 41, 12, 15, 16, 21, 55, 32, 23, 40, 27, 47, 54, 3, 8, 13, 19, 24, 26, 42, 44, 9, 45, 11, 14, 17, 20, 53, 43, 48, 49, 57], 
                  1: [62, 59, 60, 63, 99, 66, 84, 30, 72, 73, 86, 88, 89, 74, 82, 83, 96, 97, 100, 31, 75, 76, 71, 93, 101, 78, 79, 91, 94, 81, 77, 61, 98, 95, 102, 87], 
                  2: [52, 51, 58, 64, 65, 69, 68], 
                  3: [2, 0, 1, 4, 5, 6], 
                  4: [67, 103, 104]}

for key in new_cluster_dic.keys() :
  for val in new_cluster_dic[key] :
    book_pred_vals_scan[val] = key

print("NMI : ", calculateNMI(books_truth, book_pred_vals_scan))
print("ARI : ", calculateARI(books_truth, book_pred_vals_scan))
print("Purity : ", calculatePurity(books_truth, book_pred_vals_scan))


NMI :  0.4237737734704243
ARI :  0.5314393841230972
Purity :  0.8095238095238095


### Girvan Newman

* Using in built Girvan-newmann algorithm for creation of graph and then finding communities from it.

In [ ]:
books_girvan = algorithms.girvan_newman(book_graph, level=2)
print(books_girvan.communities)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 53, 54, 55, 56, 57], [28, 30, 31, 59, 60, 61, 62, 63, 66, 67, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104], [64, 65, 68, 69, 51, 52, 58]]


* Generating predicted truth values for each node relating them with a cluster index denoting the label of cluster that they belong to.

* Evaluate metrics

In [ ]:
book_pred_vals_girvan = [0]*105
new_cluster_dic = {
                    0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 53, 54, 55, 56, 57], 
                    1: [28, 30, 31, 59, 60, 61, 62, 63, 66, 67, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104], 
                    2: [64, 65, 68, 69, 51, 52, 58]
                  }

for key in new_cluster_dic.keys() :
  for val in new_cluster_dic[key] :
    book_pred_vals_girvan[val] = key

print("NMI : ", calculateNMI(books_truth, book_pred_vals_girvan))
print("ARI : ", calculateARI(books_truth, book_pred_vals_girvan))
print("Purity : ", calculatePurity(books_truth, book_pred_vals_girvan))


NMI :  0.5754215653909663
ARI :  0.6795097820780607
Purity :  0.8476190476190476
